In [14]:
# NOTE: This is a temporary notebook used to develop the processing pipeline
# while waiting for Spark cluster issues to be resolved
# It's using pandas instead of spark

In [18]:
import jupyter_black

jupyter_black.load()

ModuleNotFoundError: No module named 'jupyter_black'

In [27]:
import pandas as pd
import numpy as np

df_raw = pd.read_json("sample_data.json", lines=True)
df_raw.head(10)

,author,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,gilded,...,is_submitter,link_id,parent_id,permalink,retrieved_on,score,stickied,subreddit,subreddit_id,author_cakeday
0,Dethcola,,Clairemont,A quarry,True,0,1506816000,None,0,0,...,False,t3_73ieyz,t3_73ieyz,/r/sandiego/comments/73ieyz/best_place_for_gra...,1509189606,3,False,sandiego,t5_2qq2q,NaN
1,PennyBotV2,flair2-penny1,The Bot,[Salutations! I'm not sure what you said.](htt...,True,0,1506816000,None,0,0,...,False,t3_73g740,t1_dnqiiv7,/r/RWBY/comments/73g740/ruby_gives_greetings_a...,1509189606,3,False,RWBY,t5_2vhg0,NaN
2,Sir_Firebum,HOU-wagon,Astros Bandwagon,I got into baseball at about he same time Matt...,True,0,1506816000,None,0,0,...,False,t3_73hlwn,t1_dnqc3lu,/r/baseball/comments/73hlwn/matt_cain_in_his_f...,1509189606,2,False,baseball,t5_2qm7u,NaN
3,deanzynut,woodcutting,99 wcing btw,FUCKING TORY,True,0,1506816000,None,0,0,...,True,t3_73gw9b,t1_dnqdo99,/r/2007scape/comments/73gw9b/carls_platebody/d...,1509189606,18,False,2007scape,t5_2wbww,NaN
4,OfullOstomacheO,None,None,I see a water dragon,True,0,1506816000,None,0,0,...,False,t3_73i6z3,t3_73i6z3,/r/mildlyinteresting/comments/73i6z3/this_bric...,1509189606,1,False,mildlyinteresting,t5_2ti4h,NaN
5,PlusOn3,green,Sub 2:30 4x4 (2cep) YT: Plus Two,Wait. The Michigan what? Where is this? Is thi...,True,0,1506816000,None,0,0,...,False,t3_73g65l,t1_dnq4z9q,/r/Cubers/comments/73g65l/but_hey_why_not/dnqi...,1509189606,1,False,Cubers,t5_2r6a3,NaN
6,yeee_bot,None,None,ye fam,True,0,1506816000,None,0,0,...,False,t3_73hvr0,t1_dnqijyp,/r/teenagers/comments/73hvr0/im_hungry/dnqik1a/,1509189606,2,False,teenagers,t5_2rjli,NaN
7,grrrrreat,None,None,143417804| &gt; United States Anonymous (ID: L...,True,0,1506816000,None,0,0,...,True,t3_73dvyh,t3_73dvyh,/r/4chan4trump/comments/73dvyh/143412250who_ha...,1509189606,1,False,4chan4trump,t5_3hds7,NaN
8,psych4191,mississippistate-sheet1-row06-col08-eggbowl-2s...,Mississippi State Bulldogs / Egg Bowl,That is some chicken salad outta chicken shit ...,True,0,1506816000,None,0,0,...,False,t3_73hgz4,t3_73hgz4,/r/CFB/comments/73hgz4/game_thread_mississippi...,1509189606,2,False,CFB,t5_2qm9d,NaN
9,fishboy2000,northland,Northland,Does he even know the rules?,True,0,1506816000,None,0,0,...,False,t3_73feje,t1_dnqick7,/r/rugbyunion/comments/73feje/match_thread_pum...,1509189606,1,False,rugbyunion,t5_2qkbe,NaN


In [3]:
# Step 1: Filter out columns that will not be used

cols_to_keep = ["author", "body", "created_utc", "score", "subreddit", "subreddit_id"]

df_reddit = df_raw[cols_to_keep]
df_reddit.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
0,Dethcola,A quarry,1506816000,3,sandiego,t5_2qq2q
1,PennyBotV2,[Salutations! I'm not sure what you said.](htt...,1506816000,3,RWBY,t5_2vhg0
2,Sir_Firebum,I got into baseball at about he same time Matt...,1506816000,2,baseball,t5_2qm7u
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
5,PlusOn3,Wait. The Michigan what? Where is this? Is thi...,1506816000,1,Cubers,t5_2r6a3
6,yeee_bot,ye fam,1506816000,2,teenagers,t5_2rjli
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe


In [4]:
# Step 2.1: Count the biggest subreddits (by number of posts)

df_subred_count = pd.DataFrame(df_reddit.groupby("subreddit")["author"].count()).reset_index().rename(mapper={"author": "count"}, axis="columns")
df_subred_count.head(10)

,subreddit,count
0,1200isplenty,1
1,2007scape,22
2,2meirl4meirl,5
3,3DS,1
4,3Dprinting,3
5,3d6,1
6,3dshacks,4
7,40kLore,5
8,49ers,2
9,4PanelCringe,1


In [5]:
# Step 2.2: Get list with top 100 subreddit ids

subs_to_incl = 100

df_top_subs = df_subred_count.sort_values(by="count", ascending=False).iloc[0:subs_to_incl]
top_subs = df_top_subs["subreddit"].tolist()
print(top_subs)

['AskReddit', 'CFB', 'CrazyIdeas', 'news', 'ConciseIAmA', '4chan4trump', 'politics', 'RocketLeagueExchange', 'nba', 'The_Donald', 'edc_raffle', 'worldnews', 'funny', 'teenagers', 'leagueoflegends', 'Ice_Poseidon', 'DestinyTheGame', 'pics', 'hockey', 'nfl', 'gaming', 'videos', 'baseball', 'Jokes', 'Fireteams', 'rugbyunion', 'SquaredCircle', 'pcmasterrace', 'anime', 'relationships', 'LifeProTips', 'gifs', 'PoliticalHumor', 'fantasyfootball', 'MLS', 'Showerthoughts', 'DotA2', 'Rainbow6', 'MMA', 'mildlyinteresting', 'PUBATTLEGROUNDS', 'exmormon', 'europe', 'gonewild', 'movies', 'neoliberal', 'FireEmblemHeroes', 'aww', 'Overwatch', 'todayilearned', 'RocketLeague', 'GlobalOffensive', 'rupaulsdragrace', 'conspiracy', 'NBA2k', 'FIFA', '2007scape', 'btc', 'cars', 'soccer', 'hiphopheads', 'PS4', 'ChapoTrapHouse', 'FFBraveExvius', 'ffxiv', 'MaddenUltimateTeam', 'Guildwars2', 'CringeAnarchy', 'Bitcoin', 'magicTCG', 'food', 'sports', 'totalwar', 'destiny2', 'ColoradoRockies', 'CasualConversation', 

In [6]:
# Step 3.1: Filter data to only contain top subreddits

df_sub_filtered = df_reddit.loc[df_reddit["subreddit"].isin(top_subs)]
df_sub_filtered.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
2,Sir_Firebum,I got into baseball at about he same time Matt...,1506816000,2,baseball,t5_2qm7u
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
6,yeee_bot,ye fam,1506816000,2,teenagers,t5_2rjli
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe
10,raspberryseltzer,Tequila.,1506816000,2,CFB,t5_2qm9d
13,GonadusTwistus,you're really ignorant of history then. like ...,1506816000,2,The_Donald,t5_38unr
14,ithinkisaidtoomuch,lets see how deep this rabbit hole goes...,1506816000,1,CrazyIdeas,t5_2snxj


In [7]:
# Step 4.1: Find out who are active users

df_user_count = pd.DataFrame(df_sub_filtered.groupby("author")["body"].count()).reset_index().rename(mapper={"body": "count"}, axis="columns")
df_user_count.head(10)

,author,count
0,--Adam,1
1,-Dr-Mantis-Toboggan-,1
2,-TheDoodlebob,1
3,-damnittohell,2
4,-jb24,1
5,-redditardation-,1
6,-santman23-,1
7,-shadowstorm-,1
8,000TragicSolitude,2
9,0Catalyst,1


In [8]:
# Step 4.2: Find active and inactive users

comment_threshold = 1

df_top_users = df_user_count.loc[df_user_count["count"] > comment_threshold]
top_users = df_top_users["author"].tolist()
print(top_users)

['-damnittohell', '000TragicSolitude', '1000Airplanes', '1ain', '2crudedudes', '2minforgoaltending', '4kickz', '5225225', '69bit', '71stMB', 'AMaSTRIPPER_AMA', 'ARealKoala', 'ASmallCrane', 'AUfromthaBOOT', 'Aeolun', 'Aeschylus_', 'AfroClam', 'Amk5398', 'Archaeo', 'ArmoredSpearhead', 'AutoModerator', 'Aw_Frig', 'BASEDME7O', 'BBagot', 'BSnapZ', 'Bacxaber', 'Bamaborn97', 'BannerDay', 'Bartins', 'Beastbrook00', 'Beekeeper1987', 'BeerGogglesFTW', 'BenjaminWebb161', 'Bernie1400', 'BlueStripe100', 'Bobyyyyyyyghyh', 'BravoHotelTango', 'Brgisme', 'BucketsAMF', 'BuddyUpInATree', 'BullAlligator', 'CPGT', 'Caleb_26', 'Canux', 'CarolinaPunk', 'CaseofTrophies', 'Cheeba_Addict', 'Cheomesh', 'Chick-inn', 'ChickenwingChamp', 'Clever_Sardonic_Name', 'Clozer12', 'Cogeno', 'ColdWednesdayNight', 'Concise_AMA_Bot', 'CoolCatLovesAllKids', 'Cpt_Soban', 'CultOfMourning', 'D1amondDude', 'D4rkr4in', 'DaGoz', 'Dajja15', 'Daktush', 'Dalek1234', 'De1CawlidgeHawkey', 'DeemSleep', 'Deliwoot', 'DemonBurritoCat', 'Deni

In [9]:
# Step 4.3: Filter out inactive users with few comments

df_user_filtered = df_sub_filtered.loc[df_sub_filtered["author"].isin(top_users)]
df_user_filtered.head(10)

,author,body,created_utc,score,subreddit,subreddit_id
3,deanzynut,FUCKING TORY,1506816000,18,2007scape,t5_2wbww
4,OfullOstomacheO,I see a water dragon,1506816000,1,mildlyinteresting,t5_2ti4h
7,grrrrreat,143417804| &gt; United States Anonymous (ID: L...,1506816000,1,4chan4trump,t5_3hds7
8,psych4191,That is some chicken salad outta chicken shit ...,1506816000,2,CFB,t5_2qm9d
9,fishboy2000,Does he even know the rules?,1506816000,1,rugbyunion,t5_2qkbe
10,raspberryseltzer,Tequila.,1506816000,2,CFB,t5_2qm9d
14,ithinkisaidtoomuch,lets see how deep this rabbit hole goes...,1506816000,1,CrazyIdeas,t5_2snxj
15,Brgisme,"You are arguing to argue, I don't mind a story...",1506816000,2,NBA2k,t5_2s84e
22,halfhere,"They act like it’s some crazy idea, and the Fo...",1506816000,1,CFB,t5_2qm9d
23,AutoModerator,"***Welcome to /r/edc_raffle, please read the r...",1506816000,1,edc_raffle,t5_3jcb6


In [33]:
# Step 5.1: Get a list of all subreddits that each user has posted in

df_user_subs = pd.DataFrame(df_user_filtered.groupby("author")["subreddit"].apply(lambda x: list(np.unique(x))).reset_index())
df_user_subs.head(50)

,author,subreddit
0,-damnittohell,[The_Donald]
1,000TragicSolitude,[AskReddit]
2,1000Airplanes,[politics]
3,1ain,[CFB]
4,2crudedudes,"[nintendo, xboxone]"
5,2minforgoaltending,[hockey]
6,4kickz,[gonewild]
7,5225225,[anime]
8,69bit,[news]
9,71stMB,[AskReddit]


In [34]:
# Step 5.2: Create tuples from all those subreddits

df_user_subs["subreddit_tuples"] = df_user_subs["subreddit"].apply(lambda x: [(sub_1, sub_2) for sub_1 in x for sub_2 in x])
df_user_subs.head(10)

,author,subreddit,subreddit_tuples
0,-damnittohell,[The_Donald],"[(The_Donald, The_Donald)]"
1,000TragicSolitude,[AskReddit],"[(AskReddit, AskReddit)]"
2,1000Airplanes,[politics],"[(politics, politics)]"
3,1ain,[CFB],"[(CFB, CFB)]"
4,2crudedudes,"[nintendo, xboxone]","[(nintendo, nintendo), (nintendo, xboxone), (x..."
5,2minforgoaltending,[hockey],"[(hockey, hockey)]"
6,4kickz,[gonewild],"[(gonewild, gonewild)]"
7,5225225,[anime],"[(anime, anime)]"
8,69bit,[news],"[(news, news)]"
9,71stMB,[AskReddit],"[(AskReddit, AskReddit)]"


In [35]:
# Step 5.3: Get a list of all the tuples

all_sub_tuples = df_user_subs["subreddit_tuples"].tolist()
all_sub_tuples

[[('The_Donald', 'The_Donald')],
 [('AskReddit', 'AskReddit')],
 [('politics', 'politics')],
 [('CFB', 'CFB')],
 [('nintendo', 'nintendo'),
  ('nintendo', 'xboxone'),
  ('xboxone', 'nintendo'),
  ('xboxone', 'xboxone')],
 [('hockey', 'hockey')],
 [('gonewild', 'gonewild')],
 [('anime', 'anime')],
 [('news', 'news')],
 [('AskReddit', 'AskReddit')],
 [('aww', 'aww'), ('aww', 'funny'), ('funny', 'aww'), ('funny', 'funny')],
 [('nba', 'nba')],
 [('AskReddit', 'AskReddit')],
 [('CFB', 'CFB')],
 [('AskReddit', 'AskReddit')],
 [('CFB', 'CFB')],
 [('news', 'news')],
 [('AskReddit', 'AskReddit')],
 [('hockey', 'hockey')],
 [('CasualConversation', 'CasualConversation')],
 [('AskReddit', 'AskReddit'),
  ('AskReddit', 'GlobalOffensive'),
  ('AskReddit', 'Overwatch'),
  ('AskReddit', 'PUBATTLEGROUNDS'),
  ('AskReddit', 'RocketLeagueExchange'),
  ('AskReddit', 'Showerthoughts'),
  ('AskReddit', 'cars'),
  ('AskReddit', 'edc_raffle'),
  ('AskReddit', 'fantasyfootball'),
  ('AskReddit', 'leagueoflegen

In [49]:
# Step 5.4: Do mapreduce to get count of each tuple

import itertools

flat_tuple_list = list(itertools.chain(*all_sub_tuples))
tuple_count = map(lambda x: (x, 1), flat_tuple_list)
sorted_tuple_count = sorted(tuple_count, key=lambda x: x[0])

def reduce_func(key, values):
    counts = [x[1] for x in values]
    return [key, sum(counts)]

reduce_result = []
for key, val in itertools.groupby(sorted_tuple_count, key=lambda x: x[0]):
    reduce_result.append(reduce_func(key, list(val)))

print(reduce_result)

[[('2007scape', '2007scape'), 2], [('2007scape', 'AskReddit'), 1], [('2007scape', 'Bitcoin'), 1], [('2007scape', 'CFB'), 1], [('2007scape', 'ChapoTrapHouse'), 1], [('2007scape', 'Conservative'), 1], [('2007scape', 'CringeAnarchy'), 1], [('2007scape', 'DestinyTheGame'), 1], [('2007scape', 'DotA2'), 1], [('2007scape', 'FIFA'), 1], [('2007scape', 'Fireteams'), 1], [('2007scape', 'GlobalOffensive'), 1], [('2007scape', 'Guildwars2'), 1], [('2007scape', 'Ice_Poseidon'), 1], [('2007scape', 'Incels'), 1], [('2007scape', 'Jokes'), 1], [('2007scape', 'LifeProTips'), 1], [('2007scape', 'MLS'), 1], [('2007scape', 'MMA'), 1], [('2007scape', 'MaddenUltimateTeam'), 1], [('2007scape', 'NASCAR'), 1], [('2007scape', 'NBA2k'), 1], [('2007scape', 'NintendoSwitch'), 1], [('2007scape', 'Overwatch'), 1], [('2007scape', 'PS4'), 1], [('2007scape', 'PUBATTLEGROUNDS'), 1], [('2007scape', 'PoliticalHumor'), 1], [('2007scape', 'Rainbow6'), 1], [('2007scape', 'RocketLeagueExchange'), 1], [('2007scape', 'Showerthoug

In [56]:
# Step 5.5: Filter out entries where both tuple elements are the same

result_clean = [i for i in reduce_result if i[0][0] != i[0][1]]
sorted(result_clean, key=lambda x: x[1], reverse=True)

[[('AskReddit', 'Showerthoughts'), 5],
 [('Showerthoughts', 'AskReddit'), 5],
 [('AskReddit', 'gaming'), 3],
 [('AskReddit', 'mildlyinteresting'), 3],
 [('AskReddit', 'personalfinance'), 3],
 [('AskReddit', 'politics'), 3],
 [('AskReddit', 'relationships'), 3],
 [('gaming', 'AskReddit'), 3],
 [('mildlyinteresting', 'AskReddit'), 3],
 [('mildlyinteresting', 'personalfinance'), 3],
 [('personalfinance', 'AskReddit'), 3],
 [('personalfinance', 'mildlyinteresting'), 3],
 [('politics', 'AskReddit'), 3],
 [('relationships', 'AskReddit'), 3],
 [('AskReddit', 'GlobalOffensive'), 2],
 [('AskReddit', 'LifeProTips'), 2],
 [('AskReddit', 'Overwatch'), 2],
 [('AskReddit', 'PUBATTLEGROUNDS'), 2],
 [('AskReddit', 'RocketLeagueExchange'), 2],
 [('AskReddit', 'aww'), 2],
 [('AskReddit', 'cars'), 2],
 [('AskReddit', 'edc_raffle'), 2],
 [('AskReddit', 'fantasyfootball'), 2],
 [('AskReddit', 'funny'), 2],
 [('AskReddit', 'leagueoflegends'), 2],
 [('AskReddit', 'news'), 2],
 [('AskReddit', 'smashbros'), 2]

In [ ]:
# Step 5.6: Get rid of duplicates
# E.g. [('AskReddit', 'gaming'), 3] is a dupe of [('gaming', 'AskReddit'), 3]

# TODO